In [2]:

import pandas as pd
import regex as re
import nltk
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('93mwordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Error loading 93mwordnet: Package '93mwordnet' not found
[nltk_data]     in index
[nltk_data] Downloading package stopwords to /Users/aitor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/aitor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# cargar excel con tweets
ruta = 'modelo/ISIS_Twitter.xlsx'
df = pd.read_excel(ruta)

# convertir a lista
tweets = df.values.tolist()

df = pd.DataFrame(tweets, columns=['Fecha', 'Usuario', 'Displayed name', 'Contenido', 'label'])

print(df.head())
# Limpiar los tweets

# Eliminar filas con la columna 'Contenido' vacía
df.dropna(subset=['Contenido'], inplace=True)

# Eliminar filas duplicadas en todo el dataset
df.drop_duplicates(inplace=True)

# Restablecer el índice después de eliminar filas
df.reset_index(drop=True, inplace=True)

df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'http\S+', '', str(x)))
df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'@(\w+)', '', str(x)))
df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'#', '', str(x)))
df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'\d+', '', str(x)))
df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', str(x)))
df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# remove stopwords and lemmatize tokens using apply() method
df['Contenido'] = df['Contenido'].apply(lambda x: x.lower())
df['Contenido'] = df['Contenido'].apply(lambda x: word_tokenize(x))
df['Contenido'] = df['Contenido'].apply(lambda x: [word for word in x if word not in stop_words])
df['Contenido'] = df['Contenido'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
df['Contenido'] = df['Contenido'].apply(lambda x: ' '.join(x))

# remove timezone information using tz_localize() method
df['Fecha'] = pd.to_datetime(df['Fecha']).dt.tz_localize(None)
## Guardar los tweets limpios en un archivo Excel ##
# ruta guardado
# y guardar los tweets en un archivo Excel
filename = "ISIS_Twitter_clean.xlsx"
ruta_carpeta = 'resultados/terrorismo/'
ruta_completa = ruta_carpeta + filename
# transfiero el dataframe a un archivo excel
df.to_excel(ruta_completa, index=False)


      Fecha       Usuario Displayed name  \
0  7/4/2016      Yara_Ari   يارا العريفي   
1  7/4/2016   PansyHunks1    Pansy Hunks   
2  7/4/2016  farhan_fyzee   Farhan Fyzee   
3  7/4/2016    cydasalleh       Rashidah   
4  7/4/2016     Zolo_san1     حسن العمري   

                                           Contenido label  
0  #ISISisNotIslam Wake up people they are litera...     N  
1  RT @Juliet777777: REVEALED ISIS now occupies A...     N  
2  RT @Abunass3r: #ISIS claims to defend Islam ye...     N  
3  RT @WeTeachLifeSir_: ISIS just bombed a mosque...     N  
4  RT @xeni: Dude ISIS is bombing Muslim people i...     N  
